Imports

In [71]:
import tensorflow as tf
import pathlib
import pandas as pd
import numpy as np
import os
import io
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
import re



from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, SimpleRNN, GRU, LSTM
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam


#Importation du dataset provenant d'un fichier csv


In [72]:
# Je donne accès à colab à mon drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


création d'un data frame pandas avec le dataset


In [73]:
df = pd.read_csv('/content/drive/MyDrive/Projets_Jedha/Data/spam.csv', encoding = 'latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# EDA

In [74]:
#on vérifie de la distribution da la variable cible
df['v1'].value_counts()

,count
v1,
ham,4825
spam,747


La variable target n'est pas équilibrée.

On remplace les 2 valeurs target par 0 et 1 pour pouvoir ensuite le traiter comme des variables numériques.

Il s'agit d'un problème de classification binaire.

In [75]:
df['v1'] = df['v1'].map({'ham': 0, 'spam': 1})
df.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [76]:
#Checking missing values per column
#Pas de valeurs manquantes dans la targe (v1) et dans la colonne v2
#Moins d'1% des lignes dans les colonnes 'Unnamed 2','Unnamed 3','Unnamed 4' contiennent des valeurs
print("DF's shape: ")
print(df.shape)
print()
print("DF's count: ")
print(df.count())
print()
print("Missing values: ")
print(df.isnull().any())
print()
print("Count of missing values: ")
display(100 * df.isnull().sum())
print()
print("Percentage of missing values: ")
display(100 * df.isnull().sum() / df.shape[0])

DF's shape: 
(5572, 5)

DF's count: 
v1            5572
v2            5572
Unnamed: 2      50
Unnamed: 3      12
Unnamed: 4       6
dtype: int64

Missing values: 
v1            False
v2            False
Unnamed: 2     True
Unnamed: 3     True
Unnamed: 4     True
dtype: bool

Count of missing values: 


,0
v1,0
v2,0
Unnamed: 2,552200
Unnamed: 3,556000
Unnamed: 4,556600



Percentage of missing values: 


,0
v1,0.000000
v2,0.000000
Unnamed: 2,99.102656
Unnamed: 3,99.784637
Unnamed: 4,99.892319


# Pre-processing


Pas de valeurs manquantes dans la target (v1) et dans la colonne v2.

Moins d'1% des lignes des colonnes 'Unnamed 2', 'Unnamed 3', 'Unnamed 4' contiennent des valeurs.

In [77]:
dataset = df.iloc[:,:2]
dataset.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [79]:
# remove double spaces and spaces at the beginning and end of strings
dataset['messages_clean'] = dataset['v2'].apply(lambda x: x.replace(" +"," ").strip())
dataset.head()


,v1,v2,messages_clean
0,0,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro..."


Tokenization - Spacy

We will keep stop words in this dataset since we are dealing mostly with short messages that may be spams. Important information may be contained in the stop words. Also special characters like "!!" may be important for detecting spams.


In [80]:
#  Premier essai avec spacy
# Import English using en_core_web_sm.load()
#import spacy
#import en_core_web_sm

#nlp = en_core_web_sm.load()
#dataset['messages_tokenized'] = dataset['messages_clean'].apply(lambda x: nlp(x))
#dataset['messages_tokenized'] = dataset['messages_tokenized'].apply(lambda x:[token.lemma_ for token in x])
#dataset["clean_token"] = [" ".join(x) for x in dataset['messages_tokenized']]

import re

# fonction pour obtenir les clean tokens : mots + charactères spéciaux
def tokenizer(text):
    # mots + charactères spéciaux
    tokens = re.findall(r'\b\w+\b|[!?#@]+', text.lower())
    return tokens

# On applique la fonction sur la colonne 'messages_clean'
dataset['messages_tokenized'] = dataset['messages_clean'].apply(lambda x: tokenizer(x))
# Join de tokens`clean_token` column
dataset["clean_token"] = [" ".join(x) for x in dataset['messages_tokenized']]
dataset['clean_token'] = dataset['clean_token'].apply(lambda x: x.replace(" +"," ").strip())

dataset.head()



,v1,v2,messages_clean,messages_tokenized,clean_token
0,0,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, crazy, available, o...",go until jurong point crazy available only in ...
1,0,Ok lar... Joking wif u oni...,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]",ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...",free entry in 2 a wkly comp to win fa cup fina...
3,0,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...",u dun say so early hor u c already then say
4,0,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...",nah i don t think he goes to usf he lives arou...


Word count

In [81]:
def clean_and_tokenize(text):
    tokens = re.findall(r'\b\w+\b|[!?#@]+', text.lower())
    return tokens

# flatten the list of tokenized words
all_words = [word for message in dataset['clean_token'] for word in clean_and_tokenize(message)]

# Count the frequency of each word
word_counts = Counter(all_words)

# number of unique words
num_unique_words = len(word_counts)
num_unique_words

8728

Tokenizer

In [82]:
vocab_size = 8728

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size) # on crée une instance du tokenizer keras

tokenizer.fit_on_texts(dataset["clean_token"])

In [83]:
dataset["messages_encoded"] = tokenizer.texts_to_sequences(dataset.clean_token)

In [84]:
dataset.head()

,v1,v2,messages_clean,messages_tokenized,clean_token,messages_encoded
0,0,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, crazy, available, o...",go until jurong point crazy available only in ...,"[54, 465, 4333, 825, 740, 649, 71, 8, 1297, 94..."
1,0,Ok lar... Joking wif u oni...,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]",ok lar joking wif u oni,"[50, 341, 1464, 466, 6, 1889]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...",free entry in 2 a wkly comp to win fa cup fina...,"[52, 482, 8, 21, 4, 781, 881, 2, 185, 1890, 11..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...",u dun say so early hor u c already then say,"[6, 246, 158, 26, 380, 2920, 6, 122, 160, 61, ..."
4,0,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...",nah i don t think he goes to usf he lives arou...,"[996, 1, 86, 25, 113, 65, 483, 2, 937, 65, 189..."


In [85]:
dataset['token_count'] = dataset['messages_encoded'].apply(lambda x: len(str(x).split(' ')))
dataset.head()

,v1,v2,messages_clean,messages_tokenized,clean_token,messages_encoded,token_count
0,0,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, crazy, available, o...",go until jurong point crazy available only in ...,"[54, 465, 4333, 825, 740, 649, 71, 8, 1297, 94...",20
1,0,Ok lar... Joking wif u oni...,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]",ok lar joking wif u oni,"[50, 341, 1464, 466, 6, 1889]",6
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...",free entry in 2 a wkly comp to win fa cup fina...,"[52, 482, 8, 21, 4, 781, 881, 2, 185, 1890, 11...",33
3,0,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...",u dun say so early hor u c already then say,"[6, 246, 158, 26, 380, 2920, 6, 122, 160, 61, ...",11
4,0,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...",nah i don t think he goes to usf he lives arou...,"[996, 1, 86, 25, 113, 65, 483, 2, 937, 65, 189...",14


In [88]:
#Flatten the lists in 'messages_encoded' column
flattened_list = [index for sublist in dataset['messages_encoded'] for index in sublist]

#unique values
unique_count = len(set(flattened_list))

print("Number of unique values:", unique_count)

Number of unique values: 8707


On vérifie la distribution du nombre de tokens par observation avec .describe()


In [89]:
# On vérifie la distribution du nombre de tokens avec .describe()

dataset['token_count'].describe()

,token_count
count,5572.000000
mean,16.171213
std,11.775442
min,1.000000
25%,8.000000
50%,12.000000
75%,24.000000
max,190.000000


In [91]:
dataset['token_count'].nlargest(5)

,token_count
1084,190
1862,173
2157,119
2847,119
2433,115


75% of the dataset contains 23 or less token per observation. We are dealing with short messages that may be spams.

Padding

In [92]:
#on choisit la taille du padding en fonction de la distribution du nombre de tokens par observation avec maxlen
messages_pad = tf.keras.preprocessing.sequence.pad_sequences(dataset.messages_encoded, padding="post", maxlen=190)
messages_pad

array([[  54,  465, 4333, ...,    0,    0,    0],
       [  50,  341, 1464, ...,    0,    0,    0],
       [  52,  482,    8, ...,    0,    0,    0],
       ...,
       [8705,   64,    8, ...,    0,    0,    0],
       [   5,  494,  114, ...,    0,    0,    0],
       [2611,   68,  480, ...,    0,    0,    0]], dtype=int32)

We define x and y through tf.data.Dataset.from_tensor_slices

v1 is the target variable (Y)
messages_pad is the feature (X)

In [139]:
dataset_model = tf.data.Dataset.from_tensor_slices((messages_pad, dataset.v1))


In [140]:
dataset.shape

(5572, 7)

### Train - test split

In [141]:
take_size = int(0.7*dataset.shape[0])


dataset_train = dataset_model.take(take_size).shuffle(take_size, seed=42)
dataset_train = dataset_train.batch(64)

In [142]:
# test_set

dataset_test = dataset_model.skip(take_size)
dataset_test = dataset_model.batch(64)


In [144]:
# On jète un oeil sur notre data
for message, label in dataset_train.take(1):
    print(message, label)

tf.Tensor(
[[ 580  119  303 ...    0    0    0]
 [4836  196   42 ...    0    0    0]
 [   1   69   28 ...    0    0    0]
 ...
 [ 695  667  265 ...    0    0    0]
 [ 312  160  868 ...    0    0    0]
 [ 868   88   65 ...    0    0    0]], shape=(64, 190), dtype=int32) tf.Tensor(
[0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0], shape=(64,), dtype=int64)


# We test 1) a simple dense layer model 2) RNN 3) GRU 4) LSTM

# Finally we will choose one of this models and try to ameliorate the results by using Fasttext for text embedding.

### Model 1: dense layers

In [145]:
vocab_size = max([max(sequence) for sequence in messages_pad]) + 1
vocab_size

8708

In [215]:
model1 = Sequential([
  Embedding(vocab_size, 32, name="embedding"), # Couche d'embedding
  GlobalAveragePooling1D(), # Couche nécessaire pour pouvoir ensuite introduire les couches Dense # moyenne de chaque mot dans la phrase dans chaque dimension.
  Dense(8, activation='relu'), # Couche dense
  Dense(1, activation="sigmoid") # Couche dense de prédiction avec une fonction d'activation sigmoide car il s'agit d'un classification binaire (0 ou 1 / non spam ou spam)
])

In [216]:
model1.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])


Model 1 training

In [217]:
# Model training
model1.fit(dataset_train,
          epochs=30,
          validation_data=dataset_test)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8525 - loss: 0.4379 - val_accuracy: 0.8659 - val_loss: 0.3808
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8658 - loss: 0.3793 - val_accuracy: 0.8659 - val_loss: 0.3744
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8673 - loss: 0.3707 - val_accuracy: 0.8659 - val_loss: 0.3717
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8658 - loss: 0.3695 - val_accuracy: 0.8659 - val_loss: 0.3702
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8692 - loss: 0.3581 - val_accuracy: 0.8659 - val_loss: 0.3583
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8618 - loss: 0.3636 - val_accuracy: 0.8659 - val_loss: 0.3523
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8670 - loss: 0.3481 - val_accuracy: 0.8659 - val_loss: 0.3386
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8666 - loss: 0.3334 - val_accuracy: 0.8659 - val_l

Model 1 loss across epochs visualization:

In [218]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=model1.history.history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=model1.history.history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()

Model 1 accuracy across epochs visualization:

In [220]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=model1.history.history["accuracy"],
                    mode='lines',
                    name='accuracy'))
fig.add_trace(go.Scatter(y=model1.history.history["val_accuracy"],
                    mode='lines',
                    name='val_accuracy'))
fig.show()

In [221]:
from sklearn.metrics import classification_report


### Predictions on the train and test sets and evaluation (classification report)

In [236]:
# predictions on the train and the test sets
predictions_prob_train = model1.predict(dataset_train)

predictions_prob_test = model1.predict(dataset_test)

# Convert probabilities to class labels
#
predictions_train = (predictions_prob_train > 0.5).astype(int)
predictions_test = (predictions_prob_test > 0.5).astype(int)


# Extract the true labels from the test set
true_labels_train = np.concatenate([y for x, y in dataset_train], axis=0)


true_labels_test = np.concatenate([y for x, y in dataset_test], axis=0)

# classification report
print('Classification report Dense layers model')
report_dense_train = classification_report(true_labels_train, predictions_train, target_names=['Class 0', 'Class 1'])
print(report_dense_train)

report_dense_test = classification_report(true_labels_test, predictions_test, target_names=['Class 0', 'Class 1'])
print(report_dense_test)

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Classification report Dense layers model
              precision    recall  f1-score   support

     Class 0       0.87      0.88      0.87      3381
     Class 1       0.13      0.12      0.12       519

    accuracy                           0.78      3900
   macro avg       0.50      0.50      0.50      3900
weighted avg       0.77      0.78      0.77      3900

              precision    recall  f1-score   support

     Class 0       0.99      1.00      0.99      4825
     Class 1       1.00      0.90      0.95       747

    accuracy                           0.99      5572
   macro avg       0.99      0.95      0.97      5572
weighted avg       0.99      0.99      0.99      5572



### RNN

In [223]:
model_rnn = tf.keras.Sequential([
    Embedding(vocab_size, 32, name="embedding"),  # Embedding layer
    SimpleRNN(units=8, return_sequences=False),  # RNN, only returning the last output
    Dense(4, activation='relu'),  # Dense layer
    Dense(1, activation="sigmoid") ]) # Output layer for binary classification

In [224]:
model_rnn.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_3 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [225]:

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [226]:
# Model training
history = model_rnn.fit(dataset_train,
         epochs=30,
        validation_data=dataset_test,
        verbose=1 )

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.7317 - loss: 0.6867 - val_accuracy: 0.8659 - val_loss: 0.6274
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.8705 - loss: 0.6081 - val_accuracy: 0.8659 - val_loss: 0.5466
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.8694 - loss: 0.5216 - val_accuracy: 0.8659 - val_loss: 0.4559
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.8587 - loss: 0.4470 - val_accuracy: 0.8659 - val_loss: 0.4063
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.8693 - loss: 0.3969 - val_accuracy: 0.8659 - val_loss: 0.3954
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - accuracy: 0.8723 - loss: 0.3840 - val_accuracy: 0.8659 - val_loss: 0.3942
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step - accuracy: 0.8633 - loss: 0.3989 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - accuracy: 0.8673 - loss: 0.3915 - val_accuracy: 0.8659 - v

Loss and Accuracy  visualisation RNN model


In [227]:
fig = go.Figure()

# Accessing the 'history' dictionary from the History object
fig.add_trace(go.Scatter(y=model_rnn.history.history["loss"],
                         mode='lines',
                         name='loss'))

fig.add_trace(go.Scatter(y=model_rnn.history.history["val_loss"],
                         mode='lines',
                         name='val_loss'))

fig.show()



In [228]:

fig = go.Figure()
fig.add_trace(go.Scatter(y=model_rnn.history.history["accuracy"],
                    mode='lines',
                    name='accuracy'))
fig.add_trace(go.Scatter(y=model_rnn.history.history["val_accuracy"],
                    mode='lines',
                    name='val_accuracy'))
fig.show()


### Predictions on the train and test sets and evaluation (classification report)

In [229]:

predictions_prob_train = model_rnn.predict(dataset_train)

predictions_prob_test = model_rnn.predict(dataset_test)

predictions_train = (predictions_prob_train > 0.5).astype(int)
predictions_test = (predictions_prob_test > 0.5).astype(int)


true_labels_train = np.concatenate([y for x, y in dataset_train], axis=0)


true_labels_test = np.concatenate([y for x, y in dataset_test], axis=0)

print('Classification report train set rnn')
report_rnn_train = classification_report(true_labels_train, predictions_train, target_names=['Class 0', 'Class 1'])
print(report_rnn_train)

print('Classification report test set rnn')
report_rnn_test = classification_report(true_labels_test, predictions_test, target_names=['Class 0', 'Class 1'])
print(report_rnn_test)

61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
Classification report train set rnn
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      3381
     Class 1       0.00      0.00      0.00       519

    accuracy                           0.87      3900
   macro avg       0.43      0.50      0.46      3900
weighted avg       0.75      0.87      0.81      3900

Classification report test set rnn
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      4825
     Class 1       0.00      0.00      0.00       747

    accuracy                           0.87      5572
   macro avg       0.43      0.50      0.46      5572
weighted avg       0.75      0.87      0.80      5572



### GRU

In [230]:
#On teste un modèle avec une couche GRU

model_gru = tf.keras.Sequential([
                Embedding(vocab_size, 32, name="embedding"),
                GRU(units=8, return_sequences=False), # returns the last output
                Dense(4, activation='relu'), # a dense layer
                Dense(1, activation="sigmoid") ])   #prediction layer

In [231]:
#model_gru.compile(optimizer='adam',
#              loss=tf.keras.losses.BinaryCrossentropy(),
#              metrics=['accuracy'])

model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [232]:
# Model training
history = model_gru.fit(dataset_train,
          epochs=30,
          validation_data=dataset_test,
          verbose = 1)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 252ms/step - accuracy: 0.8142 - loss: 0.6878 - val_accuracy: 0.8659 - val_loss: 0.6714
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 211ms/step - accuracy: 0.8619 - loss: 0.6666 - val_accuracy: 0.8659 - val_loss: 0.6512
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 16s 142ms/step - accuracy: 0.8661 - loss: 0.6462 - val_accuracy: 0.8659 - val_loss: 0.6320
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 15s 218ms/step - accuracy: 0.8757 - loss: 0.6256 - val_accuracy: 0.8659 - val_loss: 0.6142
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 201ms/step - accuracy: 0.8687 - loss: 0.6093 - val_accuracy: 0.8659 - val_loss: 0.5977
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 154ms/step - accuracy: 0.8664 - loss: 0.5036 - val_accuracy: 0.8659 - val_loss: 0.3943
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 120ms/step - accuracy: 0.8688 - loss: 0.3888 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 10s 117ms/step - accuracy: 0.8616 - loss: 0.4025 - val_accur

#### Loss and accuracy visualisation GRU model


In [233]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=model_gru.history.history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=model_gru.history.history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()

In [234]:

fig = go.Figure()
fig.add_trace(go.Scatter(y=model_gru.history.history["accuracy"],
                    mode='lines',
                    name='accuracy'))
fig.add_trace(go.Scatter(y=model_gru.history.history["val_accuracy"],
                    mode='lines',
                    name='val_accuracy'))
fig.show()

### Predictions on the train and test sets and evaluation (classification report)

In [235]:
predictions_prob_train = model_gru.predict(dataset_train)

predictions_prob_test = model_gru.predict(dataset_test)

predictions_train = (predictions_prob_train > 0.5).astype(int)
predictions_test = (predictions_prob_test > 0.5).astype(int)

true_labels_train = np.concatenate([y for x, y in dataset_train], axis=0)


true_labels_test = np.concatenate([y for x, y in dataset_test], axis=0)

report_gru_train = classification_report(true_labels_train, predictions_train, target_names=['Class 0', 'Class 1'])
print('Classication report gru train set')
print(report_gru_train)
print('Classication report gru test set')
report_gru_test = classification_report(true_labels_test, predictions_test, target_names=['Class 0', 'Class 1'])
print(report_gru_test)

61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
Classication report gru train set
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      3381
     Class 1       0.00      0.00      0.00       519

    accuracy                           0.87      3900
   macro avg       0.43      0.50      0.46      3900
weighted avg       0.75      0.87      0.81      3900

Classication report gru test set
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      4825
     Class 1       0.00      0.00      0.00       747

    accuracy                           0.87      5572
   macro avg       0.43      0.50      0.46      5572
weighted avg       0.75      0.87      0.80      5572



### LSTM

In [176]:
model_lstm = tf.keras.Sequential([
                Embedding(vocab_size, 32, name="embedding"),
                LSTM(units=8, return_sequences=False),
                Dense(4, activation='relu'), # dense layer
                Dense(1, activation="sigmoid") #prediction layer
])

In [178]:
model_lstm.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [179]:
# Model training
history = model_lstm.fit(dataset_train,
          epochs=20,
          validation_data=dataset_test,
          verbose=1               )

Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - accuracy: 0.8732 - loss: 0.4893 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 20s 251ms/step - accuracy: 0.8655 - loss: 0.3954 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 15s 154ms/step - accuracy: 0.8689 - loss: 0.3888 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 12s 193ms/step - accuracy: 0.8661 - loss: 0.3938 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 17s 145ms/step - accuracy: 0.8723 - loss: 0.3822 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 10s 144ms/step - accuracy: 0.8665 - loss: 0.3935 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 11s 155ms/step - accuracy: 0.8678 - loss: 0.3907 - val_accuracy: 0.8659 - val_loss: 0.3941
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 0.8601 - loss: 0.4056 - val_accurac

#### Loss and accuracy visualisation LSTM model


In [180]:
fig = go.Figure()

    # Plot training loss
fig.add_trace(go.Scatter(y=history.history['loss'],
                              mode='lines',
                              name='Training Loss'))

fig.add_trace(go.Scatter(y=history.history['val_loss'],
                                  mode='lines',
                                  name='Validation Loss'))

fig.update_layout(title='Model Loss',
                      xaxis_title='Epochs',
                      yaxis_title='Loss')
fig.show()



In [181]:
if 'accuracy' in history.history:

    fig.add_trace(go.Scatter(y=history.history['accuracy'],
                              mode='lines',
                              name='Training Accuracy'))

    if 'val_accuracy' in history.history:
        fig.add_trace(go.Scatter(y=history.history['val_accuracy'],
                                  mode='lines',
                                  name='Validation Accuracy'))

fig.update_layout(title='Model Loss and Accuracy',
                  xaxis_title='Epochs',
                  yaxis_title='Value',
                  legend_title='Metrics')

# Show the figure
fig.show()

### Predictions on the train and test sets and evaluation (classification report)

In [237]:
predictions_prob_train = model_lstm.predict(dataset_train)

predictions_prob_test = model_lstm.predict(dataset_test)

predictions_train = (predictions_prob_train > 0.5).astype(int)
predictions_test = (predictions_prob_test > 0.5).astype(int)

true_labels_train = np.concatenate([y for x, y in dataset_train], axis=0)


true_labels_test = np.concatenate([y for x, y in dataset_test], axis=0)

report_lstm_train = classification_report(true_labels_train, predictions_train, target_names=['Class 0', 'Class 1'])

print('Classication report lstm train set')
print(report_lstm_train)

print('Classication report lstm test set')
report_lstm_test = classification_report(true_labels_test, predictions_test, target_names=['Class 0', 'Class 1'])
print(report_lstm_test)

61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
Classication report lstm train set
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      3381
     Class 1       0.00      0.00      0.00       519

    accuracy                           0.87      3900
   macro avg       0.43      0.50      0.46      3900
weighted avg       0.75      0.87      0.81      3900

Classication report lstm test set
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      4825
     Class 1       0.00      0.00      0.00       747

    accuracy                           0.87      5572
   macro avg       0.43      0.50      0.46      5572
weighted avg       0.75      0.87      0.80      5572




# Conclusions simple dense layer model, RNN, GRU, and LSTM traininings

### If we apply keras tokeniser on the corpus we have the best perfomances are from the model with only Dense layers

### Models with RNN's GRU's and LSTM's layers didn't perform well. F1 score confirms they are not able to predict well on the classes, the 'spam' label. This might be due 1) to the fact that the dataset's target variable is not balanced 2) to the fact that dataset consists of short emails that could be spam. The messages are mostly short (75% of them have 23 or fewer tokens). Therefore, the sequential information from each messagge may not be important. That is why, in this case, they are likely to overfit much more than a simple dense layer model.


# Now we will train a new model on a embedded text with fastText and not directly with keras Tokenizer



## We will create an embedding column with fastText:

### In order to improve the first model's results, we will apply word embedding from Fast Text model (trained with much larger Dataset on English language), so that we can obtain more precise information the messages. Untill now the embedding in this project has been calculated only from the tokens within this dataset through keras Tokeniser.


In [182]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296183 sha256=b404fd5ff454ebb956f8859870c206ff56498792e089b74d002b2660392460ac
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [183]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.1 MB/s eta 0:00:00


In [184]:
import re
from unidecode import unidecode
import numpy as np

### Embedding

In [185]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

--2024-12-07 11:46:44--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.96, 3.163.189.14, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G   107MB/s    in 56s     

2024-12-07 11:47:41 (76.1 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [186]:
!gunzip cc.en.300.bin.gz

In [187]:
import fasttext

model_path = 'cc.en.300.bin'
fasttext_model = fasttext.load_model(model_path)

#### Embedding function

In [188]:
def get_embeddings(text):
    return fasttext_model.get_sentence_vector(text)  # Transform text to embeddings
data = dataset
data['embeddings'] = data['clean_token'].apply(get_embeddings)

In [189]:
data['embeddings'] = data['clean_token'].apply(get_embeddings)

In [190]:
data.embeddings.head(2)

,embeddings
0,"[0.0014776621, -0.019237092, -0.037142582, 0.0..."
1,"[0.019625248, -0.04014398, -0.047746114, 0.033..."


### Train - test split

In [191]:
from sklearn.model_selection import train_test_split

In [192]:
X = np.array(data['embeddings'].tolist())  # Convert to 2D array
y = np.array(data['v1'])

In [193]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Modèle

In [194]:
#Pas besoin d'une couche embedding
model_fast_text = Sequential([
    Dense(6, activation='relu', input_shape=(X.shape[1],)),  # Specify input shape
    Dense(1, activation='sigmoid')                             # binary classification
])

# Compile the model
model_fast_text.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model_fast_text.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                     │ (None, 6)                   │           1,806 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │               7 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,813 (7.08 KB)

 Trainable params: 1,813 (7.08 KB)

 Non-trainable params: 0 (0.00 B)

In [195]:
model_fast_text = Sequential([
  Dense(8, activation='relu'), # Couche dense
  Dense(1, activation="sigmoid") # Couche dense de prédiction avec une fonction d'activation sigmoide car il s'agit d'un classification binaire (0 ou 1 / non spam ou spam)
])

In [196]:
model_fast_text.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [197]:
# Model training
# Fit the model
history = model_fast_text.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7721 - loss: 0.6298 - val_accuracy: 0.8655 - val_loss: 0.4286
Epoch 2/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8618 - loss: 0.3896 - val_accuracy: 0.8664 - val_loss: 0.2944
Epoch 3/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8712 - loss: 0.2751 - val_accuracy: 0.8969 - val_loss: 0.2284
Epoch 4/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9120 - loss: 0.2114 - val_accuracy: 0.9381 - val_loss: 0.1846
Epoch 5/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9434 - loss: 0.1673 - val_accuracy: 0.9534 - val_loss: 0.1570
Epoch 6/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9519 - loss: 0.1560 - val_accuracy: 0.9552 - val_loss: 0.1399
Epoch 7/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9550 - loss: 0.1316 - val_accuracy: 0.9587 - val_loss: 0.1275
Epoch 8/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9551 - loss: 0.1257 - val_accuracy: 0.

Model's loss and accuracy across epochs:

In [198]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=model_fast_text.history.history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=model_fast_text.history.history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()

In [199]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=model_fast_text.history.history["accuracy"],
                    mode='lines',
                    name='accuracy'))
fig.add_trace(go.Scatter(y=model_fast_text.history.history["val_accuracy"],
                    mode='lines',
                    name='val_accuracy'))
fig.show()

# Final results and conclusions

### Classification reports from all the models trained in this notebook are listed below

### The comparison between the classification reports and the loss and accuracy scores visualisations from each model training show that the best results were obtained from the last tentative consisting in:

### 1) Obtaining  an embedding column from the original messages columns from a more sophisticated model like fastText.

### 2) with this embedding from fastText model, convering the whole English language, we train a simple 2 dense layer model.

### Classication reports from each model

Classification reports from each model are provided for comparison. The model with two dense layers, trained on the word embeddings obtained with fastText, achieved the best results in the train and the test sets

In [200]:
# Make predictions on the training data
y_train_pred = (model_fast_text.predict(X_train) > 0.5).astype("int32")

# Make predictions on the test data
y_test_pred = (model_fast_text.predict(X_test) > 0.5).astype("int32")



print("Classification Report for Training Set:")
classification_report_fasttext_train = classification_report(y_train, y_train_pred)
print(classification_report_fasttext_train)

# Classification report for the testing set
print("Classification Report for Testing Set:")
classification_report_fasttext_test = classification_report(y_test, y_test_pred)
print(classification_report_fasttext_test)

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Classification Report for Training Set:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      3860
           1       0.94      0.90      0.92       597

    accuracy                           0.98      4457
   macro avg       0.96      0.94      0.95      4457
weighted avg       0.98      0.98      0.98      4457

Classification Report for Testing Set:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       965
           1       0.94      0.90      0.92       150

    accuracy                           0.98      1115
   macro avg       0.96      0.95      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [201]:
print("Dense layers model without fasttext embedding")
print("Classification Report for Training Set:")
print(report_dense_train)
print("Classification Report for Testing Set:")

print(report_dense_test)


Dense layers model without fasttext embedding
Classification Report for Training Set:
              precision    recall  f1-score   support

     Class 0       0.87      0.87      0.87      3381
     Class 1       0.14      0.13      0.13       519

    accuracy                           0.78      3900
   macro avg       0.50      0.50      0.50      3900
weighted avg       0.77      0.78      0.77      3900

Classification Report for Testing Set:
              precision    recall  f1-score   support

     Class 0       0.99      1.00      0.99      4825
     Class 1       0.99      0.94      0.96       747

    accuracy                           0.99      5572
   macro avg       0.99      0.97      0.98      5572
weighted avg       0.99      0.99      0.99      5572



In [202]:
print("RNN model")
print("Classification Report for Training Set:")
print(report_rnn_train)
print("Classification Report for Testing Set:")
print(report_rnn_test)

RNN model
Classification Report for Training Set:
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      3381
     Class 1       0.00      0.00      0.00       519

    accuracy                           0.87      3900
   macro avg       0.43      0.50      0.46      3900
weighted avg       0.75      0.87      0.81      3900

Classification Report for Testing Set:
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      4825
     Class 1       0.00      0.00      0.00       747

    accuracy                           0.87      5572
   macro avg       0.43      0.50      0.46      5572
weighted avg       0.75      0.87      0.80      5572



In [203]:
print("GRU model")
print("Classification Report for Training Set:")
print(report_gru_train)
print("Classification Report for Testing Set:")
print(report_gru_test)

GRU model
Classification Report for Training Set:
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      3381
     Class 1       0.00      0.00      0.00       519

    accuracy                           0.87      3900
   macro avg       0.43      0.50      0.46      3900
weighted avg       0.75      0.87      0.81      3900

Classification Report for Testing Set:
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      4825
     Class 1       0.00      0.00      0.00       747

    accuracy                           0.87      5572
   macro avg       0.43      0.50      0.46      5572
weighted avg       0.75      0.87      0.80      5572



In [206]:
print("LSTM model")
print("Classification Report for Training Set:")
print(report_lstm_train)
print("Classification Report for Testing Set:")
print(report_lstm_test)

LSTM model
Classification Report for Training Set:
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      3381
     Class 1       0.00      0.00      0.00       519

    accuracy                           0.87      3900
   macro avg       0.43      0.50      0.46      3900
weighted avg       0.75      0.87      0.81      3900

Classification Report for Testing Set:
              precision    recall  f1-score   support

     Class 0       0.87      1.00      0.93      4825
     Class 1       0.00      0.00      0.00       747

    accuracy                           0.87      5572
   macro avg       0.43      0.50      0.46      5572
weighted avg       0.75      0.87      0.80      5572

